# Day 9: Encoding Error

https://adventofcode.com/2020/day/9

## Part 1

In [51]:
from pathlib import Path

INPUTS = [int(x) for x in Path("input.txt").resolve().read_text().strip().split()]

Finding the valid number by these rules is mostly trivial:

1. Start iterating through inputs beginning from index 25 (equal to the preamble length).
2. Grab the chunk of inputs from the prior 25 inputs by slicing up that prior portion of the input list.
3. Run a function that determines if that number is valid by that preamble, simply be double-iterating through the set of preamble numbers and testing the addition of one to another.

In [52]:
from typing import List


def is_valid(num: int, preamble: List[int]) -> bool:
    """Returns whether `num` is valid, determined by whether any two numbers
    in `preamble` add up to that number.
    """
    for idx, pre1 in enumerate(preamble):
        for pre2 in preamble[idx + 1 :]:
            if (pre1 + pre2) == num:
                return True
    return False


def find_invalid(numbers, preamble_length=25) -> int:
    for idx, num in enumerate(numbers[preamble_length:]):
        # We iterate over a slice of the numbers starting from `preamble`'s position.
        # When enumerated, `idx` starts at 0, and we can obtain an `end` point
        # by adding the `preamble_length` to it. This gives us two useful indices
        # with which to again slice `numbers` to obtain the set of preamble numbers
        # for our current `num`.
        end = idx + preamble_length
        preamble = numbers[idx:end]
        if not is_valid(num, preamble):
            return num


invalid_num = find_invalid(INPUTS)
print(f"Invalid number: {invalid_num}")

Invalid number: 138879426


## Part 2

Bit different on this one, needing to find a contiguous set of numbers that add up to the invalid number.

I started with an assumption that the numbers generally increased over time, even if they're not perfectly sorted to begin with. Using that, I just kept testing successive sections of the inputs by summing them together.

- If the sum ended up too low, I would increase the upper bound and try again.
- If the sum was too high, I would increase the lower bound.

This eventually finds a section that sums to match `invald_num` perfectly. This then breaks out of the loop, giving us our lower and upper bounds to work with.

Finally, it's a simple matter of sorting the numbers from that slice of the inputs and then adding the min and max together (first and last elements of a sorted list).

In [53]:
lower_bound_idx, upper_bound_idx = 0, 0

while True:
    sliced = INPUTS[lower_bound_idx:upper_bound_idx]
    if upper_bound_idx >= len(INPUTS):
        print("ya goofed")
    try_total = sum(sliced)
    if try_total == invalid_num:
        break
    if try_total > invalid_num:
        lower_bound_idx += 1
    else:
        upper_bound_idx += 1

sorted_slice = sorted(sliced)
weakness = sorted_slice[0] + sorted_slice[-1]

print("Contigious set located:")
print("-" * 23)
for idx, num in enumerate(sliced, start=lower_bound_idx + 1):
    print(f"    {idx}: {num:>12}")
print("-" * 23)
print(f"  Lowest:  {sorted_slice[0]:>10}")
print(f"  Highest: {sorted_slice[-1]:>10}")
print(f"\nEncryption weakness = {weakness}")

Contigious set located:
-----------------------
    457:      6291980
    458:      6237364
    459:      5984187
    460:      6615272
    461:      6156670
    462:      6315473
    463:      6409847
    464:      6542572
    465:      7144670
    466:     11945621
    467:      8593950
    468:      9606073
    469:      7454040
    470:      7950161
    471:      8471423
    472:      9382616
    473:     17777507
-----------------------
  Lowest:     5984187
  Highest:   17777507

Encryption weakness = 23761694
